In [37]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import os

In [2]:
large_dataset_df = (
    pd.read_csv('C:/Users/oryan/Documents/esac-project/manifests/large-training-set-labelled.csv',index_col = 0)
)

In [3]:
large_dataset_df

,zooniverse_id,thumbnail_path,interaction
0,AHZ2001cnu,/mmfs1/scratch/hpc/60/oryan/thumbnails/AHZ2001...,0
1,AHZ200067f,/mmfs1/scratch/hpc/60/oryan/thumbnails/AHZ2000...,0
2,AHZ400034x,/mmfs1/scratch/hpc/60/oryan/thumbnails/AHZ4000...,0
3,AHZ100022b,/mmfs1/scratch/hpc/60/oryan/thumbnails/AHZ1000...,0
4,AHZ2001ik8,/mmfs1/scratch/hpc/60/oryan/thumbnails/AHZ2001...,0
...,...,...,...
6563,AHZ2000x6a,/mmfs1/scratch/hpc/60/oryan/thumbnails/AHZ2000...,1
6564,AHZ2000ybi,/mmfs1/scratch/hpc/60/oryan/thumbnails/AHZ2000...,1
6565,AHZ2000vix,/mmfs1/scratch/hpc/60/oryan/thumbnails/AHZ2000...,0
6566,AHZ2000k4m,/mmfs1/scratch/hpc/60/oryan/thumbnails/AHZ2000...,0


In [5]:
validation_dataset_df = pd.read_csv(r'C:/Users/oryan/Documents/esac-project/manifests/validation_set_manifest.csv', index_col = 0)

In [14]:
matching_indices = list(large_dataset_df.merge(validation_dataset_df,on='zooniverse_id',how='right').dropna().index)

In [20]:
mask = np.full(len(validation_dataset_df), True, dtype=bool)

In [22]:
mask[matching_indices] = False

In [26]:
masked_valid_df = validation_dataset_df[mask]

In [28]:
shuffle_valid_df = (
    shuffle(masked_valid_df)
    .reset_index()
    .drop(columns='index')
)

In [29]:
shuffle_valid_df

,zooniverse_id,RA,DEC,interaction
0,AHZ2001nyt,150.420640,2.397025,0
1,AHZ20007p5,149.505153,1.790612,0
2,AHZ2001k19,149.537093,2.722033,0
3,AHZ20014y0,150.617792,2.533808,0
4,AHZ20008pa,150.584606,1.922963,0
...,...,...,...,...
326,AHZ2001gyi,150.063157,2.649346,0
327,AHZ2001fii,150.341710,2.679582,0
328,AHZ2001rxe,149.962447,2.457763,1
329,AHZ2001c2g,149.466786,2.544444,0


In [30]:
shuffle_valid_df.interaction.value_counts()

0    292
1     39
Name: interaction, dtype: int64

In [35]:
local_paths_df = (
    shuffle_valid_df
    .assign(filepath = shuffle_valid_df.zooniverse_id.apply(lambda x: '/mmfs1/scratch/hpc/60/oryan/validation-set-mixed/' + str(x) + '_300_300_3.png'))
    .assign(filepath_local = shuffle_valid_df.zooniverse_id.apply(lambda x: 'C:/Users/oryan/Documents/esac-project/cutouts/hubble-training-100-valid/' + str(x) + '_300_300_3.png'))
)

In [47]:
local_paths_checked = (
    local_paths_df
    .assign(exists = local_paths_df.filepath_local.apply(lambda x: 1 if os.path.exists(x) else np.nan))
    .dropna()
    .drop(columns='filepath_local')
)

In [49]:
local_paths_export = local_paths_checked.drop(columns='exists')

In [52]:
local_paths_export.interaction.value_counts()

0    275
1     37
Name: interaction, dtype: int64

In [50]:
local_paths_export.to_csv('C:/Users/oryan/Documents/esac-project/manifests/validation-set-manifest-checked.csv')